In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf
import datetime
from datetime import date

yf.pdr_override()

pd.set_option('display.max_rows', 10)

df = pd.DataFrame();

df = pd.read_csv('TICKERS.csv')

df.columns = ['empresa','ticker1','ticker2','ticker3']

df = df.melt(id_vars=['empresa'])

df.drop(columns=['variable','empresa'],inplace=True)

df.dropna(inplace=True)

df= df.sort_values('tickers')

df.index = range(df.shape[0])

In [124]:
aux = []

#Carregar os dados

for i in range(df.shape[0]):
    aux.append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start='2021-04-01',end = '2021-05-06'));
    
df.insert(1, 'dados', aux, allow_duplicates=True);

print('======= ACABOU ===========')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [448]:
# salvar data frame

df.to_hdf('df.h5', 'obj1');

In [442]:
# carregar data frame
df = pd.DataFrame();
aux = [];
df = pd.read_hdf('df.h5', 'obj1');
for i in range(df.shape[0]-1,-1,-1):
    if df.iloc[i,1].shape[0] <= 3:
        df = df.drop(df.index[i])
df.index = range(df.shape[0])
for i in range(df.shape[0]):
    aux.append(df.iloc[i,1])

In [384]:
# atualização

from datetime import datetime

data = datetime.fromisoformat('1970-01-01')

for i in range(df.shape[0]):
    if (df.iloc[i,1].shape[0]>0):
        if (max(df.iloc[i,1].index) > data):
            data = max(df.iloc[i,1].index)

for i in range(df.shape[0]):
    df2 = df.iloc[i,1].append(web.get_data_yahoo(df.iloc[i,0]+'.sa',start=data))
    df2 = df2[~df2.index.duplicated(keep='first')]
    df.iat[i,1] = df2
    aux.append(df.iloc[i,1])
    #if aux[i].shape[0] < 3:
    #    df = df.drop(df.index[i])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [444]:
#setup 9.1
#calculo da MME9
tend = pd.DataFrame();
for i in range(df.shape[0]):
    aux[i]['MME9'] = aux[i].Close.ewm(span=9).mean().dropna()
    aux[i]["SLOPE"]="";
    for k in range (1,aux[i].shape[0]):
        tend = aux[i].MME9[k] - aux[i].MME9[k-1]
        aux[i].iat[k,7]=tend    
                
#inicializar o 1 valor de slope como 0
#for i in range(df.shape[0]):
   # if(aux[i].SLOPE[0] == True):
       # aux[i].iat[0,7]=0
#print('don')                     
            

In [395]:
for i in range(df.shape[0]):
    del aux[i]['mark_min'];
    del aux[i]['mark_max'];

In [445]:
#perca de minima
for i in range(df.shape[0]):
    aux[i]['mark_min'] = ''
    for j in range(1,aux[i].shape[0]):
       aux[i].loc[aux[i].index[j],'mark_min'] = np.where(((aux[i].Close[j] < aux[i].Low[j-1]) & (aux[i].SLOPE[j] > 0)),1,0)

In [446]:
#superação de maxima
for i in range(df.shape[0]):
    aux[i]['mark_max'] = ''
    for j in range(3,aux[i].shape[0]):
       aux[i].loc[aux[i].index[j],'mark_max'] = np.where(((aux[i].Close[j] > aux[i].High[j-1]) | (aux[i].Open[j] > aux[i].High[j-1])) & ((aux[i].mark_min[j] == 1) | (aux[i].mark_min[j-1] == 1) | (aux[i].mark_min[j-2] == 1) | (aux[i].mark_min[j-3] == 1)) ,1,0)


In [398]:
#saber quantos mark_min
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_min[j] == 1):
            #print(df.iloc[i,0])
            contador = contador + 1;
           
            
contador

# Achar os mark_min
# aux[0].loc[aux[0]['mark_min'] == 1]

191

In [399]:
#saber quantos mark_max
contador=0

for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        if(aux[i].mark_max[j] == 1):
            contador = contador + 1;
contador

140

In [418]:
#pegar todos os tickers com mark_max e as datas
tickersmax = [];
datamax = [];
tickersmin = [];
datamin = [];
for i in range(df.shape[0]):
    for j in range(aux[i].shape[0]):
        
        if (aux[i]['mark_max'][j] == 1):
            tickersmax.append(df.iloc[i,0]);
            datamax.append(aux[i].index[j]);
        if (aux[i]['mark_min'][j] == 1):
            tickersmin.append(df.iloc[i,0]);
            datamin.append(aux[i].index[j]);
#alocar os resultados em dois pd            
df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
#tirar valores duplicados ficando com o mais recente

df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
#setup 9.2 armado          
df_setup_92_armado = pd.DataFrame();
duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]

In [420]:
#tirar os valores antigos >5 dias
for i in range(df_setup_92_armado.shape[0]-1,-1,-1):
    diff = abs(df_setup_92_armado['datas'].iloc[-1] - df_setup_92_armado.datas.iloc[i]);
    if diff.days > 7:
        df_setup_92_armado = df_setup_92_armado.drop([df_setup_92_armado.index[i]])
#reindexar o indice       
df_setup_92_armado.index = range(df_setup_92_armado.shape[0])

In [460]:
#gapping up
gap_up = []
for i in range(df.shape[0]):
    if aux[i].Open[aux[i].shape[0]-1] / aux[87].Close[aux[87].shape[0]-2] > 1.01:
        gap_up.append(df.iloc[i,0])
gap_up = pd.Series(gap_up)
        
        

In [81]:
#for i in range (teste.shape[0]):
    #if teste.iloc[i,1] == duplicates:
        #print(teste.iloc[i,1])

In [73]:
#watchlist
intradayaux = []
#if (df_setup_92_armado['dados']):
  #  del df_setup_92_armado['dados']
for i in range(df_setup_92_armado.shape[0]):
    #startdate = df_setup_92_armado.iloc[i,0];
    today = date.today();
    tomorrow = date.today() + datetime.timedelta(days=1)
    intradayaux.append(web.get_data_yahoo(df_setup_92_armado.iloc[i,1]+'.sa',start= today,end = tomorrow,interval="5m"));
    
df_setup_92_armado.insert(2, 'dados', intradayaux, allow_duplicates=True);

df_setup_92_armado.index = range(df_setup_92_armado.shape[0])

print('======= ACABOU ===========')

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [252]:
#preços intraday
for i in range(df_setup_92_armado.shape[0]):
    for j in range(df.shape[0]):
        if (df_setup_92_armado.iloc[i,1]) == (df.iloc[j,0]):
            if intradayaux[i].shape[0] != 0:
                if (intradayaux[i].Close[intradayaux[i].shape[0]-1]) >= (aux[j].High[aux[j].shape[0]-1]) :
                    print (df_setup_92_armado.iloc[i,1])

BALM3
U2ST34
BALM4
CEPE5


In [234]:
teste=pd.DataFrame();
#print(df_setup_92_armado[df_setup_92_armado['tickers'] == 'ENMT4'].index.values)
df_setup_92_armado = df_setup_92_armado.drop(index=25)


In [450]:
#retorna index em dataframe
#print(df_setup_92_armado[df_setup_92_armado['tickers'] == 'ALSO3'].index.values)
print(df[df['value'] == 'CGAS5'].index.values)

[87]


In [466]:
#dataframe to zip 92armado
#lista de tickers pro profit line_terminator=','
#compression_opts = dict(method='zip',archive_name='setup92armado.csv')  
df_setup_92_armado.tickers.to_csv('setup92armado.csv', index=False,line_terminator=',')

In [465]:
#dataframe to zip gap_up
#lista de tickers pro profit line_terminator=','
#compression_opts = dict(method='zip',archive_name='gap_up.csv')  
gap_up.to_csv('gap_up.csv', index=False,line_terminator=',')

In [463]:
gap_up


0     BIDI11
1     BMYB34
2      CEBR3
3      CEBR5
4      CEBR6
       ...  
10    HONB34
11    HPQB34
12    IBMB34
13    MMMC34
14    SBUB34
Length: 15, dtype: object

In [415]:
df_setup_92_armado['datas'].iloc[-1]

Timestamp('2021-05-10 00:00:00')

In [452]:
df.iloc[87,1]

,Open,High,Low,Close,Adj Close,Volume,MME9,SLOPE,mark_min,mark_max
Date,,,,,,,,,,
2021-04-01,152.830002,152.830002,151.139999,151.139999,151.139999,300,151.139999,,,
2021-04-05,152.000000,153.759995,149.990005,153.490005,153.490005,2800,152.445558,1.305559,0,
2021-04-06,153.580002,156.259995,153.580002,154.600006,154.600006,1400,153.328529,0.88297,0,
2021-04-07,155.699997,156.000000,152.990005,152.990005,152.990005,1400,153.213853,-0.114676,0,0
2021-04-08,153.000000,155.619995,153.000000,154.990005,154.990005,500,153.742218,0.528365,0,0
...,...,...,...,...,...,...,...,...,...,...
2021-05-04,156.360001,156.360001,151.639999,153.899994,153.899994,1600,156.468506,-0.648106,0,0
2021-05-05,153.889999,156.000000,149.110001,156.000000,156.000000,4600,156.374248,-0.094258,0,0
2021-05-06,153.740005,155.720001,153.740005,154.500000,154.500000,1200,155.997620,-0.376628,0,0


In [454]:
aux[87].Open[aux[87].shape[0]-1]

153.0

In [459]:
aux[87].Close[aux[87].shape[0]-2]

154.50999450683594